In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define label mapping for classes
label_map = {
    "tetragonal": 0,
    "orthorhombic": 1,
    "trigonal": 2,
    "cubic": 3,
    "triclinic": 4,
    "monoclinic": 5,
    "hexagonal": 6
}

# Dataset Class
class XRD_Dataset(Dataset):
    def __init__(self, file_list, data_dir):
        self.file_list = file_list
        self.data_dir = data_dir

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_name, label = self.file_list[idx]
        file_name = file_name.replace('.cif', '_convolved.npz')
        file_path = os.path.join(self.data_dir, file_name)
        data = np.load(file_path)
        x = np.stack((data['x_fine'], data['y_convolved']), axis=1).astype(np.float32)
        label = label_map[label]
        return x, label

# Load the dataset
csv_path = "/ML-course/testdata/structure_info.csv"
data_dir = "/ML-course/testdata/output_data"
df = pd.read_csv(csv_path)

# Split into train and test sets
train_files, test_files = train_test_split(df[['file_name', 'cell_structure']].values, test_size=0.2, random_state=42)

train_dataset = XRD_Dataset(train_files, data_dir)
test_dataset = XRD_Dataset(test_files, data_dir)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Vision Transformer Model
from torchvision.models.vision_transformer import VisionTransformer

class XRD_ViT(nn.Module):
    def __init__(self, seq_length, num_classes):
        super(XRD_ViT, self).__init__()
        self.vit = VisionTransformer(
            image_size=seq_length,        # Sequence length as the "image size"
            patch_size=16,                # Patch size (adjust as needed)
            num_classes=num_classes,      # Number of output classes
            embed_dim=128,                # Embedding dimension
            depth=6,                      # Number of transformer layers
            num_heads=8,                  # Number of attention heads
            mlp_dim=256,                  # Dimension of the MLP in ViT
            dropout=0.1                   # Dropout rate
        )

    def forward(self, x):
        x = self.vit(x)
        return x

# Initialize model
seq_length = 1000  # Replace with actual input sequence length
num_classes = len(label_map)
model = XRD_ViT(seq_length, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        train_loss += loss.item()
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss / len(train_loader):.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")
